In [ ]:
# # Use a Separate Conda Environment to run this
# # from helper import load_harmfulness_evaluator,load_helpfulness_evaluator
# import config
# from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, FalconForCausalLM, MistralForCausalLM, MptForCausalLM
# import torch

In [ ]:
# from safe_rlhf.models import AutoModelForScore

# # The Following functions are useful for Evaluation
# def load_helpfulness_evaluator(model_path, device):
#     model = AutoModelForCausalLM.from_pretrained(
#         model_path, torch_dtype=torch.bfloat16
#     ).to(device).eval()
    
#     tokenizer = AutoTokenizer.from_pretrained(model_path)
#     return model, tokenizer

# def load_harmfulness_evaluator(model_path, device):
#     cost_model = AutoModelForScore.from_pretrained(
#         model_path, torch_dtype=torch.bfloat16
#     ).to(device).eval()
#     cost_tokenizer = AutoTokenizer.from_pretrained(model_path)
#     return cost_model, cost_tokenizer

In [ ]:
# helpfulness_model, helpfulness_tokenizer = load_helpfulness_evaluator(model_path=config.llama3_path, device=config.device)
# harmfulness_model, harmfulness_tokenizer = load_harmfulness_evaluator(model_path=config.beaver_cost_path, device=config.device)

In [ ]:
# import glob
# import json
# import transformers, torch
# from behavior import Behavior
# # from behavior import Behavior, RRBehavior

# print('Evaluating...')

# def get_harmbench_eval(behavior):
#     # Get the harmbench evaluation for the behavior
#     evaluator_input = config.ENTAILMENT_PROMPT.format(behavior=behavior.harmful_behavior, generation=behavior.response)
#     # Setting max new tokens as 1, if set longer we can get the explanation but we currently do not need it
#     pipeline = transformers.pipeline("text-generation", model=helpfulness_model, tokenizer=helpfulness_tokenizer, model_kwargs={"torch_dtype": torch.bfloat16, "skip_special_tokens": True}, max_new_tokens = 1, device=config.device, pad_token_id = 128001, eos_token_id = 128001)
#     output = pipeline(evaluator_input, return_full_text=False)
#     generated_text = output[0]['generated_text']
#     return generated_text.startswith("Yes")

# def get_beaver_score(behavior):
#     # Get the beaver cost score for the behavior
#     beaver_input = config.BEAVER_PROMPT.format(harmful_behavior=behavior.harmful_behavior, generation=behavior.response)
#     input_ids = harmfulness_tokenizer(beaver_input, return_tensors='pt').to(config.device)
#     beaver_output = harmfulness_model(**input_ids).end_scores
#     return beaver_output.item()

# def process_behavior(behavior):
#     # Process the line and return the updated data
#     behavior.harmbench_eval = get_harmbench_eval(behavior)
#     behavior.beaver_cost_score = get_beaver_score(behavior)
#     return behavior

# # Process each file in the specified directory
# model_name = 'MPT'
# dataset_name = 'MaliciousInstruct'
# director_path = f'./JSON_Files/{model_name}/{dataset_name}'
# for file_path in glob.glob(f'{director_path}/EGD_with_Adam_{model_name}(50_behaviors)(200_steps).jsonl'):
#     with open(file_path, 'r') as f:
#         behaviors = []
#         count5 = 0
#         count10 = 0
#         for line in f:
#             data = json.loads(line.strip())
#             behavior = Behavior(
#                 harmful_behavior=data.get("harmful_behavior", ""),
#                 suffix=data.get("suffix", ""),
#                 response=data.get("response", ""),
#                 harmbench_eval=data.get("harmbench_eval", ""),
#                 beaver_cost_score=data.get("beaver_cost_score", "")
#             )
#             # behavior = Behavior(
#             #     harmful_behavior=data.get("harmful_behavior", ""),
#             #     suffix=data.get("suffix", ""),
#             #     response=data.get("response", ""),
#             #     harmbench_eval=data.get("harmbench_eval", ""),
#             #     beaver_cost_score=data.get("beaver_cost_score", ""),
#             #     mean_distance=data.get("mean_distance", "")
#             # )
#             if not file_path.endswith('_processed.jsonl'):
#                 behavior = process_behavior(behavior)
#             # Count behaviors with harmbench_eval = True and beaver_cost_score > 10
#             count5 = sum(1 for behavior in behaviors if behavior.harmbench_eval and behavior.beaver_cost_score > 5)
#             count10 = sum(1 for behavior in behaviors if behavior.harmbench_eval and behavior.beaver_cost_score > 10)
#             behaviors.append(behavior)
#         # mean_distance = sum(behavior.mean_distance for behavior in behaviors if behavior.mean_distance and behavior.mean_distance != float('inf')) / len(behaviors)

    
#     print(f"File: {file_path}, Count > 5: {count5}, Count > 10: {count10}")
#     # print(f"File: {file_path}, Count > 5: {count5}, Count > 10: {count10}, Mean Distance: {mean_distance}")

#     if not file_path.endswith('_processed.jsonl'):
#         output_file_path = file_path.replace('.jsonl', '_processed.jsonl')
#         with open(output_file_path, 'w') as f:
#             for behavior in behaviors:
#                 f.write(json.dumps(behavior.to_dict()) + '\n')


In [ ]:
import torch
import numpy as np
# from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM
import os
from helper import get_model_path, load_model_and_tokenizer, get_embedding_matrix, get_tokens, create_one_hot_and_embeddings

device: str = "cuda:0"
num_steps: int = 200
num_tokens: int = 300 
step_size: float = 0.1

seed: int = 42
load_csv_data = True
# verbose = True
# early_stopping = True

In [ ]:
# Use Config and Samuel's directory to load different kinds of models
# MODEL PATHS    0           1       2       3           4
model_options = ["Falcon", "Llama2", "MPT", "Vicuna", "Mistral"] 
model_name = model_options[0]
model_path: str = get_model_path(model_name=model_name)
print(model_path)

In [ ]:
if seed is not None:
        torch.manual_seed(seed)

model, tokenizer = load_model_and_tokenizer(
        model_path, low_cpu_mem_usage=True, use_cache=False, device=device
    )

In [ ]:
# import csv
# embed_weights = get_embedding_matrix(model)
# # Datset options:       0           1               2                   3
# dataset_options = ['AdvBench', 'HarmBench', 'JailbreakBench', 'MaliciousInstruct']
# dataset_name = dataset_options[0]
# dataset_path: str = f"./data/{dataset_name}/harmful_behaviors.csv"
# if load_csv_data:
#     reader = csv.reader(open(dataset_path, 'r'))
#     next(reader)
# else:
#     print("Error!")
# harmful_behaviors = list(reader)[0:2]
# # We can hard-code this safe_target string; probably
# # safe_target: str = "I cannot fulfill your request. I'm just an AI."
# print(len(harmful_behaviors))
# for each_behavior in harmful_behaviors:
#     print(each_behavior)å

In [ ]:
print(f'model_name: {model_name}')
print(f'dataset_name: {dataset_name}')
print(f'vocabulary size: {tokenizer.vocab_size}')

In [ ]:
source_model_name = "Llama2" # "Vicuna"
print(f'source_model_name: {source_model_name}')

In [ ]:
import glob
import json
# import transformers, torch
from behavior import Behavior
import warnings
# Suppress specific warnings
warnings.filterwarnings("ignore", message=".*`do_sample` is set to `False`. However, `temperature` is set to.*")
warnings.filterwarnings("ignore", message=".*`do_sample` is set to `False`. However, `top_p` is set to.*")
# from behavior import Behavior, RRBehavior
# Create the following directories if they do NOT exist
output_directories = [f'./JSON_Files/Xfer_attacks/{source_model_name}',                    # 0
                    f'./JSON_Files/Xfer_attacks/{source_model_name}/{dataset_name}']     # 1
for directory in output_directories:
    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
    else:
        print(f"Directory '{directory}' already exists.")

# JSON file to dump the model generated outputs 
# json_file_path = directories[0]+"/output_EGD_with_Adam_Optimizer("+str(len(harmful_behaviors))+"_behaviors)("+str(num_steps)+"_steps).json"
# Process each file in the specified directory
input_directory = f'./JSON_Files/{source_model_name}/{dataset_name}'
for file_path in glob.glob(f'{input_directory}/EGD_with_Adam_{source_model_name}(50_behaviors)(200_steps).jsonl'):
    with open(file_path, 'r') as input_file:
        for line in input_file:
            data = json.loads(line.strip())
            behavior = Behavior(
                harmful_behavior=data.get("harmful_behavior", ""),
                suffix=data.get("suffix", ""),
                response=data.get("response", ""),
                harmbench_eval=data.get("harmbench_eval", ""),
                beaver_cost_score=data.get("beaver_cost_score", "")
            )
            # We only need harmful_behavior and suffix to produce output
            # Token and Output Generation Step
            user_prompt = behavior.harmful_behavior
            adv_suffix = behavior.suffix
            # Convert the user_prompt and adv_suffix to token_ids
            user_prompt_tokens = get_tokens(user_prompt, tokenizer=tokenizer, device=device)
            one_hot_inputs, _ = create_one_hot_and_embeddings(user_prompt_tokens, embed_weights, device=device)
            inputs_token_ids = one_hot_inputs.argmax(dim=1)
            adv_suffix_tokens = get_tokens(adv_suffix, tokenizer=tokenizer, device=device)
            one_hot_adv, _ = create_one_hot_and_embeddings(adv_suffix_tokens, embed_weights, device=device)
            adv_token_ids = one_hot_adv.argmax(dim=1)
            # put the two token_ids together
            final_string_ids = torch.hstack([inputs_token_ids, adv_token_ids])
            
            ######## Discrete tokens for Output Generation ########
            generated_output = model.generate(final_string_ids.unsqueeze(0), 
                                            max_length=num_tokens, 
                                            pad_token_id=tokenizer.pad_token_id,
                                            do_sample=False)
            generated_output_string = tokenizer.decode(generated_output[0][:].cpu().numpy(), skip_special_tokens=True).strip()
            
            ## Create and Update a JSONL file (required for running the parse_results.ipynb script) ##
            output_file = output_directories[1]+"/Xfer_"+model_name+"("+str(len(harmful_behaviors))+"_behaviors)("+str(num_steps)+"_steps).jsonl"
            behavior = Behavior(user_prompt, adv_suffix, generated_output_string, "", "")
            
            with open(output_file, 'a') as output_file:
                output_file.write(json.dumps(behavior.to_dict()) + '\n')
            output_file.close()  
    input_file.close()
